# Exposure to drought


A simple accepted measure for drought is the Standardized Precipitation Index (SPI). Similar to indicators 1.1 and 1.2, this indicator is not a direct measure of health impacts of drought, but rather a measure of the risk factor, which is mediated by a range of complicated causes to health. In general, the claim is that increase in this indicator represents an increased risk, which may lead directly to negative impacts on health or require adaptation measures to mitigate negative impacts.

Agnew, C.T., Using the SPI to identify drought propose SPI thresholds:

| SPI      | Category         |
|----------|------------------|
| >= -0.84 | Normal           | 
| < -0.84  | Moderate Drought |
| < -1.28  | Severe drought   | 
| < -1.65  | Extreme drought  |



Select the locations experiencing 'Severe' drought on the 3-month basis. Define the drought indicator as 'number of months in drought' for a given year.



## Literature on selection of drought index

In 2009, the World Meteorological Organization (WMO) approved the Lincoln Declaration on Drought Indices (LDDI). The LDDI recommends that "the Standardized Precipitation Index (SPI) be used to characterize the meteorological droughts around the world", in addition to other drought indices that were in use in their service. In support of this recommendation, it was suggested that a "comprehensive user manual" describing the SPI should be developed. The manual provides a description of the index, the computation methods, specific examples of where it is currently being used, the strengths and limitations and mapping capabilities.

https://www.ncl.ucar.edu/Applications/spi.shtml

https://climatedataguide.ucar.edu/climate-data/standardized-precipitation-index-spi


Note this one also good for extreme rain

https://link.springer.com/chapter/10.1007/978-94-015-9265-9_2

## SPI drought index

>TODO formula

Python code:

https://www.drought.gov/drought/python-climate-indices


Also an implementation in NCL
https://www.ncl.ucar.edu/Applications/spi.shtml

Could apply directly to source files

https://www.ncl.ucar.edu/Document/Functions/Built-in/dim_spi_n.shtml


> NOTE the implementation from drought.gov looks modern and good (built with Numba, which suggests they know what they are doing and like modern approaches which should make life easier). Should be able to run directly on netCDF files from ECMWF.


## Data for SPI

Monthly summary data for PPT

NOTE that I have already a 1949-2012 summary of SPI. Do I really need to build my own? The main advantage is to ensure coherance across indicators and that they can be easily updated.



In [2]:
%matplotlib inline

from pathlib import Path
from datetime import date

import rasterio
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from scipy import stats
from tqdm import tnrange, tqdm_notebook


import weather_ecmwf
import population_tools

from config import DATA_SRC, WEATHER_SRC

In [6]:
PPT_FILES = WEATHER_SRC / 'ecmwf' / 'monthly_precipitation'

# Initial calculation using the already generated SPI file for 1949-2012

This could be good to deliver initial values this year, although it would be best to adapt to using the ECMWF dataset to be able to update regularly

In [ ]:
spi_file = DATA_SRC / 'weather' / 'spi3_6_12_1deg_cru_ts_3_21_1949_2012.nc'
droughts = spi.spi3.where(spi.spi3 < -1.28)

In [ ]:
def get_drought_projection(droughts):
    with population_tools.PopulationProjector() as pop:
        years = list(range(2000,2013))

        droughts_pop = xr.concat((pop.project(year, droughts.sel(year=year)) for year in years)
                                 , dim='time')
        pop_sum = pop.data.population.sum(dim=['latitude', 'longitude'])
        return droughts_pop.compute(), pop_sum.compute()
    
droughts_pop, pop_sum = get_drought_projection(droughts)

In [ ]:
droughts_pop_ts = (droughts_pop / pop_sum).sum(dim=['latitude', 'longitude'])


# Apply SPI calculation to ECMWF monthly precipitation data

In [7]:
weather_ecmwf.weather_mfdataset(PPT_FOLDER)

TypeError: Error: /Users/jonathanchambers/Data/weather/ecmwf/monthly_precipitation/1980_precipitation.nc is not a valid NetCDF 3 file
            If this is a NetCDF4 file, you may need to install the
            netcdf4 library, e.g.,

            $ pip install netcdf4
            